In [2]:
import pandas as pd
import plotly.express as px

genre_mood = pd.read_csv(r'./Dataset/songs_url_genre_mood.csv', encoding='latin1')

genre_mood.head()


,year,song,artist,play_count,Genre,Mood,spotify_url
0,2014,Thinking out Loud,Ed Sheeran,473,Pop,Romantic,https://open.spotify.com/track/34gCuhDGsG4bRPI...
1,2014,Come Away With Me,Norah Jones,242,Jazz,Calm,https://open.spotify.com/track/6jGnykaS6TkWp15...
2,2014,Happy Little Pill,Troye Sivan,242,Pop,Sad,https://open.spotify.com/track/4z8Ll8nU1jMqB8H...
3,2014,Superheroes,The Script,232,Pop Rock,Empowering,https://open.spotify.com/track/37sINbJZcFdHFAs...
4,2014,How Long Will I Love You,Ellie Goulding,189,Pop,Romantic,https://open.spotify.com/track/3X003bO0MMxXEsJ...


In [3]:
genre_mood.columns

Index(['year', 'song', 'artist', 'play_count', 'Genre', 'Mood', 'spotify_url'], dtype='object')

In [42]:
import plotly.express as px

# Assuming genre_mood is your DataFrame
fig = px.pie(genre_mood, names='Genre', title='Genre Distribution')

# Turn the plot into dark mode
fig.update_layout(
    paper_bgcolor='black',  # Set background to black
    plot_bgcolor='black',   # Set plot background to black
    font_color='white',     # Change font color to white
    title_font_color='white', # Change title font color to white
    title_x=0.5,  # Center the title
    margin=dict(t=40, b=0, l=0, r=0),  # Add margins to prevent cutting off label
    showlegend=False
)

# Show genre names directly on the pie chart
fig.update_traces(textinfo='percent+label', textfont_size=8, pull=[0.1, 0.1]) 

# Adjust width of the chart
fig.update_layout(width=280, height=330)  # Adjust width to your preference

fig.write_html("./static/charts/genre_distr.html")

# Display the pie chart
fig.show()


In [5]:
# Group by genre and sum play counts
genre_total = genre_mood.groupby('Genre')['play_count'].sum().reset_index()

# Sort by play count in descending order
genre_total = genre_total.sort_values(by='play_count', ascending=False)

# Create a bar chart for top genres
fig = px.bar(genre_total, 
             x='Genre', 
             y='play_count', 
             title='Top Genres by Total Play Count (2014-2024)', 
             labels={'play_count': 'Total Play Count', 'Genre': 'Genre'},
             color='Genre', 
             color_discrete_sequence=px.colors.qualitative.Set3)

# Apply dark mode, center the title
fig.update_layout(
    template='plotly_dark',  # Apply dark mode
    title={'x': 0.5},         # Center the title
    width=400,                
    height=500,
    xaxis={'tickangle': 45}                
)

# Show the plot
fig.show()


In [41]:
import plotly.express as px
import pandas as pd

# Group by year and genre, summing up play counts
genre_yearly = genre_mood.groupby(['year', 'Genre'])['play_count'].sum().reset_index()

# Ensure genres are sorted in ascending order
genre_yearly['Genre'] = pd.Categorical(genre_yearly['Genre'], categories=sorted(genre_yearly['Genre'].unique()), ordered=True)

# Create a stacked bar chart with dark mode
fig = px.bar(
    genre_yearly,
    x='year',
    y='play_count',
    color='Genre',
    title='Genre Distribution Over Time',
    labels={'play_count': 'Total Play Count', 'year': 'Year'},
    color_discrete_sequence=px.colors.qualitative.Bold
)

# Update layout for dark mode, centered title, and additional styling
fig.update_layout(
    template='plotly_dark',  # Apply dark theme
    title={'text': 'Genre Distribution Over Time', 'x': 0.5},  # Center title
    xaxis_title='Year',
    yaxis_title='Total Play Count',
    legend_title='Genre',  # Set a title for the legend
    width=280,
    height=330,
    margin=dict(t=40, b=10, l=10, r=10),
    showlegend=False
)

fig.write_html("./static/charts/genre_year.html")

# Show the plot
fig.show()


In [60]:
# Group by year and mood, summing up play counts
mood_yearly = genre_mood.groupby(['year', 'Mood'])['play_count'].sum().reset_index()

# Create a stacked bar chart
fig = px.bar(
    mood_yearly, 
    x='year', 
    y='play_count', 
    color='Mood', 
    title='Tracking My Mood by Top Songs (2014-2024)',
    labels={'play_count': 'Total Play Count', 'year': 'Year'},
    color_discrete_sequence=px.colors.qualitative.Bold,
    template='plotly_dark',  # Apply dark mode
    width=550,
    height=330,
)

# Center the title
fig.update_layout(
    title={'x': 0.5},
    margin=dict(t=40, b=0, l=0, r=0),
    legend=dict(
        x=1,            # Position to the right of the plot
        xanchor='left', # Anchor legend's left side to x=1
        y=0.5,          # Vertically center the legend
        yanchor='middle', # Anchor legend's middle to y=0.5
        traceorder='normal',  # Preserve legend item order
    )
)  

fig.write_html("./static/charts/mood_year.html")

# Show the plot
fig.show()


# Heatmap
1) Axis
    - x axis represents mood
    - y axis represent genres

2) Cells
    - the color intensity of the cell indicates the magnitude of the play count for that specific combination of mood and genre

3) Color Scale 
    - Lighter color (yellow): lower play counts
    - Darker color (blue): higher play counts

4) Insight 
    - Based on the color scale, pop songs are the dominating genre in my music consumption across 2014 to 2024. I am a girly pop. 
    - Pop genre has the darkest blue under 'empowering' and 'sad' mood, followed by 'romantic' and 'happy'
    - I listen to a lot of pop, indie, hip-hop and rock songs when I am angry or irritated
    - I associate calmness with jazz, indie and R&B songs.
    - I listen to a lot of hip-hop to feel empowered and motivated.
    - I tune in to R&B and Indie when I'm feeling affectionate.
    - Jazz songs are the only genre that I link exclusively to a calm mood.


In [51]:
# Create a pivot table for Genre vs Mood
genre_mood_pivot = genre_mood.pivot_table(
    index='Genre', 
    columns='Mood', 
    values='play_count', 
    aggfunc='sum', 
    fill_value=0
)

# Create a heatmap with dark mode and centered title
fig = px.imshow(
    genre_mood_pivot.values, 
    labels=dict(x='Mood', y='Genre', color='Play Count'),
    x=genre_mood_pivot.columns,
    y=genre_mood_pivot.index,
    color_continuous_scale='YlGnBu', 
    title='Genre vs Mood'
)

# Apply dark mode and center the title
fig.update_layout(
    template='plotly_dark',  # Apply dark mode
    title={'x': 0.5},         # Center the title
    width=280,
    height=330,
    showlegend=False,
    coloraxis_showscale=False,
    margin=dict(t=40, b=0, l=0, r=0),
    xaxis=dict(
        tickangle=45,  # Tilt x-axis labels by 45 degrees
        tickfont=dict(size=8)  # Adjust font size for x-axis
    ),
    yaxis=dict(
        tickfont=dict(size=8)  # Adjust font size for y-axis
    ) 
)

fig.write_html("./static/charts/heatmap.html")

# Show the plot
fig.show()


In [37]:
# Group by year and genre, summing up play counts
genre_trends = genre_mood.groupby(['year', 'Genre'])['play_count'].sum().reset_index()

# Create a line chart
fig = px.line(genre_trends, 
              x='year', 
              y='play_count', 
              color='Genre', 
              title='Genre Trends', 
              labels={'play_count': 'Total Play Count', 'year': 'Year'})

# Apply dark mode and center the title
fig.update_layout(
    template='plotly_dark',  # Apply dark mode
    title={'x': 0.5},         # Center the title
    width=280,                
    height=330,      
    margin=dict(t=40, b=10, l=10, r=10),  
    showlegend=False        
)

fig.write_html("./static/charts/genre_trend.html")

# Show the plot
fig.show()


In [69]:
import plotly.express as px

# Create a scatter plot with facet rows and dark mode
fig = px.scatter(genre_mood, 
                 x='year', 
                 y='play_count', 
                 color='Genre', 
                 hover_name='song', 
                 title='Interactive Song Rankings by Genre',
                 labels={'play_count': 'Play Count', 'year': 'Year'},
                 facet_col='Genre',  # For faceting by Genre
                 facet_col_wrap=3)   # This will split into 3 columns

# Apply dark mode and adjust layout
fig.update_layout(
    template='plotly_dark',  # Apply dark mode
    title={'text': 'Interactive Song Rankings by Genre', 'x': 0.5},  # Center the title
    showlegend=False,  # Hide the legend
    height=460,  # Adjust height if necessary
    width=850,  # Adjust width for better layout (wider layout for 3 columns)
    margin=dict(t=60, b=10, l=10, r=10),  
)

# Remove y-axis titles for all facets
fig.update_yaxes(title=None)

fig.write_html("./static/charts/song_ranking_genre.html")

fig.show()
